In [1]:
import numpy as np
from scipy.io import loadmat
import tensorflow as tf
import glob
import time
import matplotlib.pyplot as plt
%matplotlib inline

print("import done")

import done


In [2]:
def plot_eeg(data):
    plt.figure(figsize=(10,20))
    for i in range(0,16):
        plt.subplot(8,2,i+1)
        plt.plot(data[:,i])
    
    plt.savefig('foo.pdf', bbox_inches='tight')

In [3]:
def mat2tfr(p_file):
    #converting mat file as numpy
    mat = loadmat(p_file)
    data = mat['dataStruct']['data'][0][0]

    #getting the filename and retrieving the patient, segement and label data
    pat, seg, label = p_file.split('/')[-1].split('.')[0].split("_")

    dstfolder = "./dataset/"
    dstfilename = dstfolder + pat + "_" + seg + "_" + label + ".tfr"
    print("Converting " + p_file + " ----> " + dstfilename)

    #plot_eeg(data)
    
    #creating the TensorFlow Records 
    tfrwriter = tf.python_io.TFRecordWriter(dstfilename)
    
    #reshape and transform to list due to TFRecords
    data = np.reshape(data, -1).tolist()

    #filling protobuff
    protobuf = tf.train.Example(
        features=tf.train.Features(feature={
            'label':  tf.train.Feature(int64_list = tf.train.Int64List(value= [int(label)])),
            #'data' :  tf.train.Feature(float_list = tf.train.FloatList(value= data))
    }))

    tfrwriter.write(protobuf.SerializeToString())
    tfrwriter.close()
    

def dataset2tfr(folder):
    filenames = glob.glob(folder)
    print("Converting #%d files."%len(filenames))
    
    for files in filenames:
        t = time.time()
        
        mat2tfr(files)

        elapsed = time.time() - t
        print("elapsed: %.3fs"%elapsed)
        
        
dataset2tfr("./data/train_1/1_1_*.mat")

print('finished')
        

Converting #2 files.
Converting ./data/train_1/1_1_0.mat ----> ./dataset/1_1_0.tfr
elapsed: 0.269s
Converting ./data/train_1/1_1_1.mat ----> ./dataset/1_1_1.tfr
elapsed: 0.153s
finished


In [7]:
slim = tf.contrib.slim

with tf.Graph().as_default():
    filenames = glob.glob("./dataset/*.tfr")
    print("Loading #%d files."%len(filenames))
    
    reader = tf.TFRecordReader
    
    keys_to_features = {
        'label': tf.FixedLenFeature([], tf.int64),
        #'data': tf.FixedLenFeature([], tf.float32),
    }
    items_to_handlers = {
      'label': slim.tfexample_decoder.Tensor('label'),
      #'data': slim.tfexample_decoder.Tensor('data'),
    }    
    decoder = slim.tfexample_decoder.TFExampleDecoder(
        keys_to_features, items_to_handlers)
    
    items_to_descriptions = {
        'label': 'Label indicating preictal and interictal.', 
        #'data': '240000 sample points of iEEG.',
    }
    
    dataset = slim.dataset.Dataset(
        data_sources=filenames, 
        reader=reader, 
        decoder=decoder, 
        num_samples=1, 
        items_to_descriptions=items_to_descriptions)
    
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, shuffle=True, num_epochs=None, common_queue_capacity=16, common_queue_min=1)
    
    labels = data_provider.get(['label'])
    
    # Batch it up.
    batch_size=2
    labels = tf.train.batch(labels, batch_size=batch_size, num_threads=1, capacity=2 * batch_size)
    
    with tf.Session() as sess:    
        with slim.queues.QueueRunners(sess):
            np_labels = sess.run([labels])
            print(np_labels)
    
print('done')
    

Loading #2 files.
[array([1, 0])]
done
